In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow

In [ ]:
import os
import numpy as np
import pickle
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten

In [ ]:
# GPU setup with minimal memory growth and precision
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        try:
            tf.config.experimental.set_memory_growth(device, True)
            # Set a memory limit to reduce load
            tf.config.set_logical_device_configuration(
                device, [tf.config.LogicalDeviceConfiguration(memory_limit=4096)]  # Example: 4GB
            )
            print(f"GPU configured with a memory limit of 4096 MB.")
        except Exception as e:
            print(f"Error configuring GPU: {e}")
else:
    print("No GPU detected, running on CPU.")

try:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled for speedup.")
except ValueError:
    print("Mixed precision not supported, running with default precision.")

GPU configured with a memory limit of 4096 MB.
Mixed precision enabled for speedup.


In [ ]:
# Function to save progress
def save_progress(filepath, data):
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)

# Function to load progress
def load_progress(filepath):
    if os.path.exists(filepath):
        try:
            with open(filepath, 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            print(f"Error loading checkpoint {filepath}: {e}")
    return []


In [ ]:
# Function to load frames and extract landmarks
# Placeholder for actual landmark extraction (use a library like Dlib or Mediapipe in practice)
def extract_landmarks_from_frames(frames):
    mp_face_mesh = mp.solutions.face_mesh
    landmark_list = []

    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True) as face_mesh:
        for frame in frames:
            results = face_mesh.process(frame)
            if results.multi_face_landmarks:
                # Extract 468 3D landmarks for the detected face
                face_landmarks = results.multi_face_landmarks[0]
                landmarks = [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark]  # Normalize to 3D
                landmark_list.append(landmarks)
            else:
                # Append None or zeros if no face is detected
                landmark_list.append([(0, 0, 0)] * 468)

    return landmark_list

# Function to process videos and save landmarks with checkpoints
def extract_landmark_features_with_checkpoint(video_frames_path, checkpoint_path, total_videos):
    saved_progress = load_progress(checkpoint_path)
    start_index = len(saved_progress)

    if start_index >= total_videos:
        print(f"All videos already processed for {checkpoint_path}")
        return np.array(saved_progress)

    progress_bar = tqdm(total=total_videos, desc="Extracting Landmark Features")
    progress_bar.update(start_index)

    for video_idx in range(start_index, total_videos):
        video_path = video_frames_path[video_idx]
        try:
            frames = load_video_frames(video_path)  # Function to load frames from a video
            landmarks = extract_landmarks_from_frames(frames)
            saved_progress.append(landmarks)
        except Exception as e:
            print(f"Error processing video {video_idx}: {e}")
            continue
        save_progress(checkpoint_path, saved_progress)
        progress_bar.update(1)

    progress_bar.close()
    return np.array(saved_progress)

In [ ]:
# Function to load video frames (replace this with actual logic for your dataset)
def load_video_frames(video_path):
    # Dummy example: Replace this with code to load frames from videos
    return [np.random.rand(224, 224, 3) for _ in range(48)]  # Simulated 30 frames per video

In [ ]:
if __name__ == "__main__":
    # Define paths and directories
    base_path = 'drive/MyDrive/final_dataset/features/'
    os.makedirs(base_path, exist_ok=True)

    video_frames_real_path = 'drive/MyDrive/final_dataset/cropped_frames/real/'
    video_frames_fake_path = 'drive/MyDrive/final_dataset/cropped_frames/fake/'

    real_checkpoint = os.path.join(base_path, 'landmarks_real.pkl')
    fake_checkpoint = os.path.join(base_path, 'landmarks_fake.pkl')

    # Get list of video frame directories
    real_video_paths = [os.path.join(video_frames_real_path, d) for d in os.listdir(video_frames_real_path)]
    fake_video_paths = [os.path.join(video_frames_fake_path, d) for d in os.listdir(video_frames_fake_path)]

    total_real_videos = len(real_video_paths)
    total_fake_videos = len(fake_video_paths)

    # Extract landmark features
    print("Processing real videos...")
    landmark_features_real = extract_landmark_features_with_checkpoint(real_video_paths, real_checkpoint, total_real_videos)

    print("Processing fake videos...")
    landmark_features_fake = extract_landmark_features_with_checkpoint(fake_video_paths, fake_checkpoint, total_fake_videos)

    print("Landmark feature extraction completed.")


Processing real videos...


Extracting Landmark Features: 100%|██████████| 967/967 [05:55<00:00,  2.72it/s]


Processing fake videos...


Extracting Landmark Features: 100%|██████████| 967/967 [05:36<00:00,  2.88it/s]

Landmark feature extraction completed.
